### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL


In [4]:

import requests
import time

# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # Create query with a page number
    query_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q=love&fq=section_name:Movies&sort=newest&page={page}&api-key={os.getenv('NYT_API_KEY')}"

    try:
        # Make a "GET" request and retrieve the JSON
        response = requests.get(query_url)
        reviews = response.json()

        # Add a twelve second interval between queries to stay within API query limits
        time.sleep(12)

        # Try and save the reviews to the reviews_list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        
        # Print the page that was just retrieved
        print(f"Page {page} retrieved successfully.")

    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"No results for page {page}. Error: {e}")
        break


In [5]:

# Preview the first 5 results in JSON format
formatted_tmdb_movies = json.dumps(tmdb_movies_list[:5], indent=4)
print(formatted_tmdb_movies)


In [6]:

import pandas as pd
from pandas import json_normalize

# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = json_normalize(reviews_list)

# Extract the title from the "headline.main" column and save it to a new column "title"
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(lambda x: extract_keywords(x) if isinstance(x, list) else "")

# Create a list from the "title" column using to_list()
titles_list = reviews_df['title'].to_list()


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string


In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database


### Access The Movie Database API

In [10]:

import time

# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + os.getenv("TMDB_API_KEY")

# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Sleeping for 1 second after 50 requests")
        time.sleep(1)
    
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    query_url = f"{url}{title}{tmdb_key_string}"
    response = requests.get(query_url)
    movie_data = response.json()

    # Include a try clause to search for the full movie details.
    try:
        # Get movie id
        movie_id = movie_data['results'][0]['id']

        # Make a request for the full movie details
        movie_detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={os.getenv('TMDB_API_KEY')}&language=en-US"
        movie_details = requests.get(movie_detail_url).json()

        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            'title': title,
            'original_title': movie_details['original_title'],
            'budget': movie_details['budget'],
            'original_language': movie_details['original_language'],
            'homepage': movie_details['homepage'],
            'overview': movie_details['overview'],
            'popularity': movie_details['popularity'],
            'runtime': movie_details['runtime'],
            'revenue': movie_details['revenue'],
            'release_date': movie_details['release_date'],
            'vote_average': movie_details['vote_average'],
            'vote_count': movie_details['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        })
        
        # Print out the title that was found
        print(f"Found movie details for: {title}")
    
    except Exception as e:
        # Print out a statement if a movie is not found
        print(f"Movie not found for title: {title}. Error: {e}")


In [11]:
# Create an empty list to store the results


# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



In [12]:

# Preview the first 5 results in JSON format
formatted_tmdb_movies = json.dumps(tmdb_movies_list[:5], indent=4)
print(formatted_tmdb_movies)


In [13]:

# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on="title")

# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "")

# Display the fixed DataFrame
print(merged_df.head())


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [16]:

# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"], errors='ignore')

# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

# Export data to CSV without the index
merged_df.to_csv("merged_movie_data.csv", index=False)


In [17]:
# Delete duplicate rows and reset index


In [18]:
# Export data to CSV without the index
